In [9]:
class_map = pd.read_csv('yamnet_class_map.csv').set_index('display_name').to_dict()['mid']
print(class_map)

{'Speech': '/m/09x0r', 'Child speech, kid speaking': '/m/0ytgt', 'Conversation': '/m/01h8n0', 'Narration, monologue': '/m/02qldy', 'Babbling': '/m/0261r1', 'Speech synthesizer': '/m/0brhx', 'Shout': '/m/07p6fty', 'Bellow': '/m/07q4ntr', 'Whoop': '/m/07rwj3x', 'Yell': '/m/07sr1lc', 'Children shouting': '/t/dd00135', 'Screaming': '/m/03qc9zr', 'Whispering': '/m/02rtxlg', 'Laughter': '/m/01j3sz', 'Baby laughter': '/t/dd00001', 'Giggle': '/m/07r660_', 'Snicker': '/m/07s04w4', 'Belly laugh': '/m/07sq110', 'Chuckle, chortle': '/m/07rgt08', 'Crying, sobbing': '/m/0463cq4', 'Baby cry, infant cry': '/t/dd00002', 'Whimper': '/m/07qz6j3', 'Wail, moan': '/m/07qw_06', 'Sigh': '/m/07plz5l', 'Singing': '/m/015lz1', 'Choir': '/m/0l14jd', 'Yodeling': '/m/01swy6', 'Chant': '/m/02bk07', 'Mantra': '/m/01c194', 'Child singing': '/t/dd00005', 'Synthetic singing': '/t/dd00006', 'Rapping': '/m/06bxc', 'Humming': '/m/02fxyj', 'Groan': '/m/07s2xch', 'Grunt': '/m/07r4k75', 'Whistling': '/m/01w250', 'Breathing': 

In [13]:
print(mid_to_index)


{'/m/07pggtn': 0, '/m/01jg02': 1, '/t/dd00067': 2, '/m/06ncr': 3, '/m/0dgbq': 4, '/m/0395lw': 5, '/m/03dnzn': 6, '/m/07szfh9': 7, '/m/015lz1': 8, '/m/02rhddq': 9, '/m/04szw': 10, '/m/01v1d8': 11, '/m/07rn7sz': 12, '/m/02fsn': 13, '/m/0fqfqc': 14, '/m/03cczk': 15, '/m/03gvt': 16, '/m/07r81j2': 17, '/m/07qn5dc': 18, '/m/0b_fwt': 19, '/m/0c3f7m': 20, '/m/01v_m0': 21, '/m/07s12q4': 22, '/m/02p3nc': 23, '/m/07plct2': 24, '/t/dd00035': 25, '/m/06hps': 26, '/m/07pczhz': 27, '/m/07qfgpx': 28, '/m/07prgkl': 29, '/m/07rv4dm': 30, '/m/05rwpb': 31, '/m/019jd': 32, '/m/01g90h': 33, '/m/0ytgt': 34, '/m/04fq5q': 35, '/m/0l7xg': 36, '/m/04cvmfc': 37, '/m/0164x2': 38, '/m/07rc7d9': 39, '/m/01glhc': 40, '/m/01rd7k': 41, '/m/0h0rv': 42, '/m/0hsrw': 43, '/m/05kq4': 44, '/m/05r5c': 45, '/m/0jb2l': 46, '/m/01lyv': 47, '/m/01swy6': 48, '/m/04s8yn': 49, '/m/05rj2': 50, '/m/07bgp': 51, '/t/dd00136': 52, '/m/07cx4': 53, '/m/015p6': 54, '/m/03kmc9': 55, '/m/046dlr': 56, '/m/07s02z0': 57, '/m/07rrlb6': 58, '/m/02

In [18]:
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"Number of GPUs: {torch.cuda.device_count()}")


CUDA Available: True
Number of GPUs: 1


In [2]:
import torch
torch.cuda.empty_cache()  # Clear memory cache after each training step or epoch



In [3]:
print(f"Memory allocated: {torch.cuda.memory_allocated()} bytes")
print(f"Memory cached: {torch.cuda.memory_reserved()} bytes")

Memory allocated: 0 bytes
Memory cached: 0 bytes


In [2]:
torch.cuda.empty_cache()  # Clears GPU cache


NameError: name 'torch' is not defined

In [1]:
import glob
import os
import json
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2Processor
import torch
import librosa
from collections import namedtuple
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # Set to block CUDA errors

# Load the class map CSV (mapping from mid to index)
class_map_df = pd.read_csv('yamnet_class_map.csv')
class_map = pd.read_csv('yamnet_class_map.csv').set_index('display_name').to_dict()['mid']

# Create a mapping from mid (string) to index (integer)
mid_to_index = {mid: idx for idx, mid in enumerate(set(class_map.values()))}

# Initialize the model and processor
model = Wav2Vec2ForSequenceClassification.from_pretrained('facebook/wav2vec2-base-960h', num_labels=521)

processor = Wav2Vec2Processor.from_pretrained('facebook/wav2vec2-base-960h')

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("Moved model to GPU if available")

# Define a namedtuple for dataset items
AudioSample = namedtuple("AudioSample", ["input_values", "labels"])

class AudioDataset(Dataset):
    def __init__(self, audio_directory, ontology_file, mid_to_index):
        with open(ontology_file, 'r') as f:
            self.ontology_data = json.load(f)

        self.mid_to_index = mid_to_index
        self.audio_directory = audio_directory
        self.audio_files = glob.glob(os.path.join(self.audio_directory, '**', '*.wav'), recursive=True)
        
        # Populate the dataset by calling prepare_data
        self.data = self.prepare_data()

    def prepare_data(self):
        data = []
        for category in self.ontology_data:
            if "positive_examples" in category:
                category_name = category["name"]
                mid = category["id"]  # Get the mid for the current category

                # Use the mid to get the index from the mid_to_index
                if mid in self.mid_to_index:
                    label = self.mid_to_index[mid]  # Get the integer index as the label
                else:
                    label = -1  # Default to -1 if not found

                for audio_file in self.audio_files:
                    if category_name.lower() in audio_file.lower():
                        audio_file = audio_file.replace("\\", "/")
                        data.append({"audio": audio_file, "label": label})
        return data
    
    def load_audio(self, file_path):
        """Load and preprocess audio using Wav2Vec2Processor."""
        try:
            if not os.path.isfile(file_path):
                raise FileNotFoundError(f"WAV file not found: {file_path}")

            # Load audio using librosa and resample to 16kHz
            audio_data, sr = librosa.load(file_path, sr=16000)
            print(f"Successfully loaded audio: {file_path}, shape: {audio_data.shape}, dtype: {audio_data.dtype}")

            # Preprocess using Wav2Vec2Processor
            inputs = processor(audio_data, sampling_rate=sr, return_tensors="pt", padding=True)
            processed_audio = inputs.input_values.squeeze(0)  # Remove batch dimension

            print(f"Processed audio shape: {processed_audio.shape}")

            # Return processed audio
            return processed_audio

        except Exception as e:
            print(f"Error loading audio file {file_path}: {e}")
            return None

    def __getitem__(self, idx):
        """Get one item (audio, label) for the dataset."""
        sample = self.data[idx]
        audio_data = self.load_audio(sample["audio"])
        label = sample["label"]

        # Ensure audio_data is valid
        if audio_data is None:
            print(f"Error loading audio at index {idx}, returning dummy data.")
            return {"input_values": torch.zeros(1), "labels": torch.tensor(label, dtype=torch.long)}

        # Trim or pad audio to max_length
        max_length = 160000  # Set a max_length for padding/truncating
        if audio_data.shape[0] < max_length:
            padding = torch.zeros(max_length - audio_data.shape[0])
            audio_data = torch.cat([audio_data, padding])
        else:
            audio_data = audio_data[:max_length]

        print(f"Successfully loaded audio at index {idx}, shape: {audio_data.shape}, label: {label}")
        
        return {"input_values": audio_data.clone().detach(), "labels": torch.tensor(label, dtype=torch.long)}

    def __len__(self):
        return len(self.data)

# Initialize the dataset and dataloaders
audio_directory = r"audiosets/ontology"
ontology_file = 'ontology.json'

# Initialize dataset and prepare data
dataset = AudioDataset(audio_directory, ontology_file, mid_to_index)

# Now split the dataset into train and test sets (80% train, 20% test)
train_data, test_data = train_test_split(dataset.data, test_size=0.2, random_state=42)

# Initialize train and test datasets using the split data
train_dataset = AudioDataset(audio_directory, ontology_file, mid_to_index)
test_dataset = AudioDataset(audio_directory, ontology_file, mid_to_index)

# Assign the split data to the datasets
train_dataset.data = train_data
test_dataset.data = test_data

# Define the compute_metrics function
def compute_metrics(pred):
    logits, labels = pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = (predictions == labels).mean()
    return {'accuracy': accuracy}

# Training setup
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=3,
    save_steps=10,
    disable_tqdm=False,
    report_to="tensorboard",
    load_best_model_at_end=True,
)

# Initialize train and test dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)

# Pass the datasets without specifying the dataloaders in Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Training loop with handling for -1 labels
for epoch in range(int(training_args.num_train_epochs)):
    print(f"Training epoch {epoch + 1}")
    model.train()  # Set model to training mode
    for batch in train_dataloader:
        input_values = batch['input_values'].to(device)
        labels = batch['labels'].to(device)

        # Skip batches where label is -1
        valid_indices = labels != -1
        if valid_indices.sum() == 0:
            continue  # Skip this batch if all labels are -1

        # Only select valid indices
        input_values = input_values[valid_indices]
        labels = labels[valid_indices]

        # Forward pass
        outputs = model(input_values, labels=labels)
        loss = outputs.loss
        loss.backward()  # Backward pass to compute gradients

        # Clear GPU cache to prevent memory overflow
        torch.cuda.empty_cache()

    # Run evaluation after each epoch (optional)
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        for batch in test_dataloader:
            input_values = batch['input_values'].to(device)
            labels = batch['labels'].to(device)

            # Skip batches where label is -1
            valid_indices = labels != -1
            if valid_indices.sum() == 0:
                continue  # Skip this batch if all labels are -1

            # Only select valid indices
            input_values = input_values[valid_indices]
            labels = labels[valid_indices]

            outputs = model(input_values, labels=labels)
            loss = outputs.loss
            logits = outputs.logits
            print(f"Evaluation loss: {loss.item()}")

    torch.cuda.empty_cache()  # Clear GPU cache after each epoch


Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Moved model to GPU if available


c:\Users\prave\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Training epoch 1
Successfully loaded audio: audiosets/ontology/Electric guitar_4.wav, shape: (2106143,), dtype: float32
Processed audio shape: torch.Size([2106143])
Successfully loaded audio at index 1324, shape: torch.Size([160000]), label: 47
Successfully loaded audio: audiosets/ontology/Howl (wind)_2.wav, shape: (160125,), dtype: float32
Processed audio shape: torch.Size([160125])
Successfully loaded audio at index 530, shape: torch.Size([160000]), label: 87
Successfully loaded audio: audiosets/ontology/Waves, surf_4.wav, shape: (159754,), dtype: float32
Processed audio shape: torch.Size([159754])
Successfully loaded audio at index 2218, shape: torch.Size([160000]), label: 105
Successfully loaded audio: audiosets/ontology/Dog_5.wav, shape: (1473074,), dtype: float32
Processed audio shape: torch.Size([1473074])
Successfully loaded audio at index 1756, shape: torch.Size([160000]), label: 19
Successfully loaded audio: audiosets/ontology/Howl_3.wav, shape: (160125,), dtype: float32
Proc

KeyboardInterrupt: 

In [2]:

from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm  # Optional: for progress bar
import glob
import os
import json
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2Processor
import torch
import librosa
from collections import namedtuple
import os

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # Set to block CUDA errors

# Load the class map CSV (mapping from mid to index)
class_map_df = pd.read_csv('yamnet_class_map.csv')
class_map = pd.read_csv('yamnet_class_map.csv').set_index('display_name').to_dict()['mid']

# Create a mapping from mid (string) to index (integer)
mid_to_index = {mid: idx for idx, mid in enumerate(set(class_map.values()))}

# Initialize the model and processor
model = Wav2Vec2ForSequenceClassification.from_pretrained('facebook/wav2vec2-base-960h', num_labels=521)

processor = Wav2Vec2Processor.from_pretrained('facebook/wav2vec2-base-960h')

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("Moved model to GPU if available")


# Define a namedtuple for dataset items
AudioSample = namedtuple("AudioSample", ["input_values", "labels"])

class AudioDataset(Dataset):
    def __init__(self, audio_directory, ontology_file, mid_to_index):
        with open(ontology_file, 'r') as f:
            self.ontology_data = json.load(f)

        self.mid_to_index = mid_to_index
        self.audio_directory = audio_directory
        self.audio_files = glob.glob(os.path.join(self.audio_directory, '**', '*.wav'), recursive=True)
        
        # Populate the dataset by calling prepare_data
        self.data = self.prepare_data()

    def prepare_data(self):
        data = []
        for category in self.ontology_data:
            if "positive_examples" in category:
                category_name = category["name"]
                mid = category["id"]  # Get the mid for the current category

                # Use the mid to get the index from the mid_to_index
                if mid in self.mid_to_index:
                    label = self.mid_to_index[mid]  # Get the integer index as the label
                else:
                    label = -1  # Default to -1 if not found

                for audio_file in self.audio_files:
                    if category_name.lower() in audio_file.lower():
                        audio_file = audio_file.replace("\\", "/")
                        data.append({"audio": audio_file, "label": label})
        return data
    
    def load_audio(self, file_path):
        """Load and preprocess audio using Wav2Vec2Processor."""
        try:
            if not os.path.isfile(file_path):
                raise FileNotFoundError(f"WAV file not found: {file_path}")

            # Load audio using librosa and resample to 16kHz
            audio_data, sr = librosa.load(file_path, sr=16000)
            print(f"Successfully loaded audio: {file_path}, shape: {audio_data.shape}, dtype: {audio_data.dtype}")

            # Preprocess using Wav2Vec2Processor
            inputs = processor(audio_data, sampling_rate=sr, return_tensors="pt", padding=True)
            processed_audio = inputs.input_values.squeeze(0)  # Remove batch dimension

            print(f"Processed audio shape: {processed_audio.shape}")

            # Return processed audio
            return processed_audio

        except Exception as e:
            print(f"Error loading audio file {file_path}: {e}")
            return None

    def __getitem__(self, idx):
        """Get one item (audio, label) for the dataset."""
        sample = self.data[idx]
        audio_data = self.load_audio(sample["audio"])
        label = sample["label"]

        # Ensure audio_data is valid
        if audio_data is None:
            print(f"Error loading audio at index {idx}, returning dummy data.")
            return {"input_values": torch.zeros(1), "labels": torch.tensor(label, dtype=torch.long)}

        # Trim or pad audio to max_length
        max_length = 160000  # Set a max_length for padding/truncating
        if audio_data.shape[0] < max_length:
            padding = torch.zeros(max_length - audio_data.shape[0])
            audio_data = torch.cat([audio_data, padding])
        else:
            audio_data = audio_data[:max_length]

        print(f"Successfully loaded audio at index {idx}, shape: {audio_data.shape}, label: {label}")
        
        return {"input_values": audio_data.clone().detach(), "labels": torch.tensor(label, dtype=torch.long)}

    def __len__(self):
        return len(self.data)

# Initialize the dataset and dataloaders
audio_directory = r"audiosets/ontology"
ontology_file = 'ontology.json'

# Initialize dataset and prepare data
dataset = AudioDataset(audio_directory, ontology_file, mid_to_index)

# Now split the dataset into train and test sets (80% train, 20% test)
train_data, test_data = train_test_split(dataset.data, test_size=0.2, random_state=42)

# Initialize train and test datasets using the split data
train_dataset = AudioDataset(audio_directory, ontology_file, mid_to_index)
test_dataset = AudioDataset(audio_directory, ontology_file, mid_to_index)

# Assign the split data to the datasets
train_dataset.data = train_data
test_dataset.data = test_data

# Define the compute_metrics function
def compute_metrics(pred):
    logits, labels = pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = (predictions == labels).mean()
    return {'accuracy': accuracy}

# Training setup
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=3,
    save_steps=10,
    disable_tqdm=False,
    report_to="tensorboard",
    load_best_model_at_end=True,
)

# Initialize train and test dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)

# Function to evaluate the model on the test set
def evaluate_model(model, test_dataset, device):
    model.eval()  # Set model to evaluation mode
    all_labels = []
    all_preds = []
    
    # Iterate over test dataset
    with torch.no_grad():  # Disable gradient calculation during evaluation
        for batch in tqdm(test_dataset, desc="Evaluating"):
            input_values = batch['input_values'].to(device)
            labels = batch['labels'].to(device)
            
            # Skip batches where label is -1
            valid_indices = labels != -1
            if valid_indices.sum() == 0:
                continue  # Skip this batch if all labels are -1

            # Only select valid indices
            input_values = input_values[valid_indices]
            labels = labels[valid_indices]
            
            # Forward pass
            outputs = model(input_values)
            logits = outputs.logits
            
            # Get predictions (highest logits)
            predictions = torch.argmax(logits, dim=-1)
            
            # Append to the lists
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predictions.cpu().numpy())
    
    # Compute evaluation metrics
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')  # You can change the average type ('micro', 'macro', 'weighted')
    
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"F1-Score (weighted): {f1:.4f}")
    
    return accuracy, f1

# Evaluate the model on the test dataset
accuracy, f1 = evaluate_model(model, test_dataset, device)


Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Moved model to GPU if available


c:\Users\prave\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Evaluating:   0%|          | 1/610 [00:04<46:03,  4.54s/it]

Successfully loaded audio: audiosets/ontology/Harpsichord_2.wav, shape: (160125,), dtype: float32
Processed audio shape: torch.Size([160125])
Successfully loaded audio at index 0, shape: torch.Size([160000]), label: -1
Successfully loaded audio: audiosets/ontology/Dishes, pots, and pans_6.wav, shape: (4317054,), dtype: float32
Processed audio shape: torch.Size([4317054])
Successfully loaded audio at index 1, shape: torch.Size([160000]), label: 237


Evaluating:   0%|          | 3/610 [00:05<13:16,  1.31s/it]

...

Successfully loaded audio: audiosets/ontology/Change ringing (campanology)_4.wav, shape: (160125,), dtype: float32
Processed audio shape: torch.Size([160125])
Successfully loaded audio at index 603, shape: torch.Size([160000]), label: 114
Successfully loaded audio: audiosets/ontology/Fart_3.wav, shape: (160125,), dtype: float32
Processed audio shape: torch.Size([160125])
Successfully loaded audio at index 604, shape: torch.Size([160000]), label: 458
Successfully loaded audio: audiosets/ontology/Hoot_0.wav, shape: (160125,), dtype: float32
Processed audio shape: torch.Size([160125])
Successfully loaded audio at index 605, shape: torch.Size([160000]), label: 17


Evaluating: 100%|█████████▉| 608/610 [02:43<00:00,  9.59it/s]

Successfully loaded audio: audiosets/ontology/Sliding door_1.wav, shape: (146008,), dtype: float32
Processed audio shape: torch.Size([146008])
Successfully loaded audio at index 606, shape: torch.Size([160000]), label: 217
Successfully loaded audio: audiosets/ontology/Saxophone_4.wav, shape: (160125,), dtype: float32
Processed audio shape: torch.Size([160125])
Successfully loaded audio at index 607, shape: torch.Size([160000]), label: 34
Successfully loaded audio: audiosets/ontology/Typewriter_0.wav, shape: (10036965,), dtype: float32
Processed audio shape: torch.Size([10036965])
Successfully loaded audio at index 608, shape: torch.Size([160000]), label: 366
Successfully loaded audio: audiosets/ontology/Gargling_4.wav, shape: (159754,), dtype: float32
Processed audio shape: torch.Size([159754])
Successfully loaded audio at index 609, shape: torch.Size([160000]), label: 354


Evaluating: 100%|██████████| 610/610 [02:45<00:00,  3.70it/s]

Accuracy: 0.17%
F1-Score (weighted): 0.0001


In [ ]:
import torch
import librosa
from transformers import Wav2Vec2Processor, Wav2Vec2ForSequenceClassification
import numpy as np

# Load the pre-trained model and processor
model = Wav2Vec2ForSequenceClassification.from_pretrained('facebook/wav2vec2-base-960h', num_labels=521)
processor = Wav2Vec2Processor.from_pretrained('facebook/wav2vec2-base-960h')

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def predict_audio(file_path):
    """Predict the class of an audio file."""
    
    # Step 1: Load the audio file
    audio_data, sr = librosa.load(file_path, sr=16000)  # Ensure the sample rate is 16kHz as required by the model
    
    # Step 2: Preprocess audio with the Wav2Vec2 processor
    inputs = processor(audio_data, sampling_rate=sr, return_tensors="pt", padding=True)
    
    # Move inputs to the same device as the model
    input_values = inputs.input_values.to(device)

    # Step 3: Perform inference
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        # Get the model's output (logits)
        outputs = model(input_values)
        logits = outputs.logits

    # Step 4: Get the predicted class (index of highest logit)
    predicted_class_idx = torch.argmax(logits, dim=-1).item()

    # You can map the predicted index to a class label if needed
    print(f"Predicted class index: {predicted_class_idx}")
    
    return predicted_class_idx

# Example usage
file_path = "test.wav"
predicted_class_idx = predict_audio(file_path)

# Assuming mid_to_index is the original mapping from class name (or MID) to index
# Reverse the mapping to go from index to class name
index_to_mid = {v: k for k, v in mid_to_index.items()}

# Get the class name from the predicted class index
predicted_mid_name = index_to_mid.get(predicted_class_idx, "Unknown")
print(f"Predicted class name: {predicted_mid_name}")

# Load the class map CSV (mapping from mid to index)
# Load the class map CSV (mapping from mid to display_name)
class_map_df = pd.read_csv('yamnet_class_map.csv')

# Create a mapping from 'mid' to 'display_name'
mid_to_display_name = class_map_df.set_index('mid')['display_name'].to_dict()

# Example: Get the display name from a given mid

display_name = mid_to_display_name.get(predicted_m  id_name, "Unknown_mid")
print(f"Display name for {predicted_mid_name}: {display_name}")
class_map = pd.read_csv('yamnet_class_map.csv').set_index('display_name').to_dict()['mid']

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Predicted class index: 417
Predicted class name: /m/01kcd
Display name for /m/01kcd: Brass instrument


In [1]:
import glob
import os
import json
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2Processor
import torch
import librosa
#from torch.nn.utils.rnn import pad_sequence
from collections import namedtuple
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# Load the class map CSV (mapping from mid to index)
class_map_df = pd.read_csv('yamnet_class_map.csv')
class_map = pd.read_csv('yamnet_class_map.csv').set_index('display_name').to_dict()['mid']

# Create a mapping from mid (string) to index (integer)
mid_to_index = {mid: idx for idx, mid in enumerate(set(class_map.values()))}

# Initialize the model and processor
model = Wav2Vec2ForSequenceClassification.from_pretrained('facebook/wav2vec2-base-960h', num_labels=521)

processor = Wav2Vec2Processor.from_pretrained('facebook/wav2vec2-base-960h')

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("Moved model to GPU if available")

# Define a namedtuple for dataset items
AudioSample = namedtuple("AudioSample", ["input_values", "labels"])

class AudioDataset(Dataset):
    def __init__(self, audio_directory, ontology_file, mid_to_index):
        with open(ontology_file, 'r') as f:
            self.ontology_data = json.load(f)

        self.mid_to_index = mid_to_index
        self.audio_directory = audio_directory
        self.audio_files = glob.glob(os.path.join(self.audio_directory, '**', '*.wav'), recursive=True)
        
        # Populate the dataset by calling prepare_data
        self.data = self.prepare_data()

    def prepare_data(self):
        data = []
        for category in self.ontology_data:
            if "positive_examples" in category:
                category_name = category["name"]
                mid = category["id"]  # Get the mid for the current category

                # Use the mid to get the index from the mid_to_index
                if mid in self.mid_to_index:
                    label = self.mid_to_index[mid]  # Get the integer index as the label
                else:
                    label = -1  # Default to -1 if not found

                for audio_file in self.audio_files:
                    if category_name.lower() in audio_file.lower():
                        audio_file = audio_file.replace("\\", "/")
                        data.append({"audio": audio_file, "label": label})
        return data
    
    def load_audio(self, file_path):
        """Load and preprocess audio using Wav2Vec2Processor."""
        try:
            if not os.path.isfile(file_path):
                raise FileNotFoundError(f"WAV file not found: {file_path}")

            # Load audio using librosa and resample to 16kHz
            audio_data, sr = librosa.load(file_path, sr=16000)
            print(f"Successfully loaded audio: {file_path}, shape: {audio_data.shape}, dtype: {audio_data.dtype}")

            # Preprocess using Wav2Vec2Processor
            inputs = processor(audio_data, sampling_rate=sr, return_tensors="pt", padding=True)
            processed_audio = inputs.input_values.squeeze(0)  # Remove batch dimension

            print(f"Processed audio shape: {processed_audio.shape}")

            # Return processed audio
            return processed_audio

        except Exception as e:
            print(f"Error loading audio file {file_path}: {e}")
            return None

    def __getitem__(self, idx):
        """Get one item (audio, label) for the dataset."""
        sample = self.data[idx]
        audio_data = self.load_audio(sample["audio"])
        label = sample["label"]

        # Ensure audio_data is valid
        if audio_data is None:
            print(f"Error loading audio at index {idx}, returning dummy data.")
            return {"input_values": torch.zeros(1), "labels": torch.tensor(label, dtype=torch.long)}

        # Trim or pad audio to max_length
        max_length = 160000  # Set a max_length for padding/truncating
        if audio_data.shape[0] < max_length:
            padding = torch.zeros(max_length - audio_data.shape[0])
            audio_data = torch.cat([audio_data, padding])
        else:
            audio_data = audio_data[:max_length]

        print(f"Successfully loaded audio at index {idx}, shape: {audio_data.shape}, label: {label}")
        
        return {"input_values": audio_data.clone().detach(), "labels": torch.tensor(label, dtype=torch.long)}

    def __len__(self):
        return len(self.data)

# Initialize the dataset and dataloaders
audio_directory = r"audiosets/ontology"
ontology_file = 'ontology.json'

# Initialize dataset and prepare data
dataset = AudioDataset(audio_directory, ontology_file, mid_to_index)

# Now split the dataset into train and test sets (80% train, 20% test)
train_data, test_data = train_test_split(dataset.data, test_size=0.2, random_state=42)

# Initialize train and test datasets using the split data
train_dataset = AudioDataset(audio_directory, ontology_file, mid_to_index)
test_dataset = AudioDataset(audio_directory, ontology_file, mid_to_index)

# Assign the split data to the datasets
train_dataset.data = train_data
test_dataset.data = test_data

# Define the compute_metrics function
def compute_metrics(pred):
    logits, labels = pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = (predictions == labels).mean()
    return {'accuracy': accuracy}

# Training setup
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=3,
    save_steps=10,
    disable_tqdm=False,
    report_to="tensorboard",
    load_best_model_at_end=True,
)

# Initialize train and test dataloaders with the custom collate_fn
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)

# Pass the datasets without specifying the dataloaders in Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Start training
trainer.train()


KeyboardInterrupt: 

In [15]:
# Take one sample from the dataset
sample = train_dataset[0]  # Take one sample
input_values = sample['input_values'].unsqueeze(0)  # Add batch dimension
labels = sample['labels'].unsqueeze(0)  # Add batch dimension

# Ensure the label is within bounds
assert 0 <= labels.item() <= 520, f"Label {labels.item()} out of bounds!"

# Pass input and labels to the model
outputs = model(input_values, labels=labels)  # Make sure to pass the labels

# The output contains 'loss' and 'logits' (among other things)
loss = outputs.loss  # Access the loss
logits = outputs.logits  # Access the logits

print(f"Loss for the sample: {loss.item()}")
print(f"Logits: {logits}")


Successfully loaded audio: audiosets/ontology/Wind instrument, woodwind instrument_2.wav, shape: (159754,), dtype: float32
Processed audio shape: torch.Size([159754])
Successfully loaded audio at index 0, shape: torch.Size([160000]), dtype: torch.float32
Loss for the sample: 6.1997270584106445
Logits: tensor([[-0.1580, -0.1213, -0.0112,  0.0322, -0.0616, -0.0335, -0.0438,  0.1859,
          0.0470,  0.0446,  0.0949, -0.0042,  0.0030, -0.1470,  0.0284, -0.1508,
          0.0313, -0.0552,  0.0296, -0.0260,  0.2222, -0.0082,  0.0016, -0.0531,
         -0.0294,  0.0156, -0.0054,  0.1366,  0.0968,  0.0012, -0.0760,  0.0263,
         -0.0250,  0.1603, -0.0762, -0.0318,  0.0198, -0.1156, -0.1020, -0.0716,
         -0.0972, -0.0438, -0.0795, -0.1077,  0.0280, -0.1169, -0.0596,  0.0981,
         -0.1082,  0.0638, -0.0292, -0.0571,  0.0552, -0.0542,  0.0653, -0.0605,
         -0.0336,  0.0578,  0.0866, -0.0536,  0.0024, -0.0595, -0.0607,  0.0596,
         -0.0023, -0.1483,  0.0867, -0.1273, -0.0

In [13]:
import glob
import os
import json
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2Processor
import torch
import librosa
from torch.nn.utils.rnn import pad_sequence
import os
#os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# Load the class map CSV (mapping from mid to index)
class_map_df = pd.read_csv('yamnet_class_map.csv')
class_map = pd.read_csv('yamnet_class_map.csv').set_index('display_name').to_dict()['mid']

# Create a mapping from mid (string) to index (integer)
#mid_to_index = class_map_df.set_index('mid').index.tolist()
mid_to_index = {mid: idx for idx, mid in enumerate(set(class_map.values()))}

# Initialize the model and processor
#model = Wav2Vec2ForSequenceClassification.from_pretrained('facebook/wav2vec2-large-960h', num_labels=len(mid_to_index))
model = Wav2Vec2ForSequenceClassification.from_pretrained('facebook/wav2vec2-large-960h', num_labels=521)

#model =  model.to_bettertransformer()

processor = Wav2Vec2Processor.from_pretrained('facebook/wav2vec2-large-960h')

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
model.to(device)
print("Moved model to GPU if available")
from collections import namedtuple

# Define a namedtuple for dataset items
AudioSample = namedtuple("AudioSample", ["input_values", "labels"])


def collate_fn( batch, max_length=160000):  # Set a max_length that works for your use case
    """Collate function to handle padding and truncation."""
    audio_data = [item["input_values"] for item in batch]
    labels = [item["labels"] for item in batch]

    # Pad or truncate audio sequences to the specified max_length
    audio_data_padded = []
    for audio in audio_data:
        if audio.shape[0] < max_length:
            # Pad if audio is shorter than max_length
            padding = torch.zeros(max_length - audio.shape[0])
            audio_data_padded.append(torch.cat([audio, padding]))
        else:
            # Truncate if audio is longer than max_length
            audio_data_padded.append(audio[:max_length])

    # Stack the labels into a tensor
    labels = torch.stack(labels, dim=0)

    # Stack the audio data into a batch tensor
    audio_data_padded = torch.stack(audio_data_padded, dim=0)

    return {"input_values": audio_data_padded, "labels": labels}


class AudioDataset(Dataset):
    def __init__(self, audio_directory, ontology_file, mid_to_index):
        with open(ontology_file, 'r') as f:
            self.ontology_data = json.load(f)

        self.mid_to_index = mid_to_index
        self.audio_directory = audio_directory
        self.audio_files = glob.glob(os.path.join(self.audio_directory, '**', '*.wav'), recursive=True)
        
        # Populate the dataset by calling prepare_data
        self.data = self.prepare_data()

    def prepare_data(self):
        data = []
        for category in self.ontology_data:
            if "positive_examples" in category:
                category_name = category["name"]
                mid = category["id"]  # Get the mid for the current category

                # Use the mid to get the index from the mid_to_index
                if mid in self.mid_to_index:
                    label = self.mid_to_index[mid]  # Get the integer index as the label
                else:
                    label = -1  # Default to -1 if not found

                for audio_file in self.audio_files:
                    if category_name.lower() in audio_file.lower():
                        audio_file = audio_file.replace("\\", "/")
                        data.append({"audio": audio_file, "label": label})
        return data
    
    def load_audio(self, file_path):
        """Load and preprocess audio using Wav2Vec2Processor."""
        try:
            if not os.path.isfile(file_path):
                raise FileNotFoundError(f"WAV file not found: {file_path}")

            # Load audio using librosa and resample to 16kHz
            audio_data, sr = librosa.load(file_path, sr=16000)
            print(f"Successfully loaded audio: {file_path}, shape: {audio_data.shape}, dtype: {audio_data.dtype}")

            # Preprocess using Wav2Vec2Processor
            inputs = processor(audio_data, sampling_rate=sr, return_tensors="pt", padding=True)
            processed_audio = inputs.input_values.squeeze(0)  # Remove batch dimension

            print(f"Processed audio shape: {processed_audio.shape}")

            # Return processed audio
            return processed_audio

        except Exception as e:
            print(f"Error loading audio file {file_path}: {e}")
            return None


        except Exception as e:
            print(f"Error loading audio file {file_path}: {e}")
            return None

    def __getitem__(self, idx):
        """Get one item (audio, label) for the dataset."""
        sample = self.data[idx]
        audio_data = self.load_audio(sample["audio"])
        label = sample["label"]

        max_length=160000
        if audio_data.shape[0] < max_length:
            # Pad if audio is shorter than max_length
            padding = torch.zeros(max_length - audio_data.shape[0])
            audio_data = torch.cat([audio_data, padding])
        else:
            # Truncate if audio is longer than max_length
            audio_data = audio_data[:max_length]
            # Ensure audio_data is valid
            
        if audio_data is None:
            print(f"Error loading audio at index {idx}, returning dummy data.")
            return {"input_values": torch.zeros(1), "labels": torch.tensor(label, dtype=torch.long)}

        # Check if audio_data is valid (numpy array or tensor)
        if isinstance(audio_data, (np.ndarray, torch.Tensor)):
            print(f"Successfully loaded audio at index {idx}, shape: {audio_data.shape}, dtype: {audio_data.dtype}")
            
            # Return valid audio data as a dictionary
            return {"input_values": audio_data.clone().detach(), "labels": torch.tensor(label, dtype=torch.long)}
        else:
            print(f"Unexpected audio_data type at index {idx}: {type(audio_data)}, returning dummy data.")
            return {"input_values": torch.zeros(1), "labels": torch.tensor(label, dtype=torch.long)}


    def __len__(self):
        return len(self.data)
    
    def collate_fn(self, batch, max_length=160000):  # Set a max_length that works for your use case
        """Collate function to handle padding and truncation."""
        audio_data = [item["input_values"] for item in batch]
        labels = [item["labels"] for item in batch]

        # Pad or truncate audio sequences to the specified max_length
        audio_data_padded = []
        for audio in audio_data:
            if audio.shape[0] < max_length:
                # Pad if audio is shorter than max_length
                padding = torch.zeros(max_length - audio.shape[0])
                audio_data_padded.append(torch.cat([audio, padding]))
            else:
                # Truncate if audio is longer than max_length
                audio_data_padded.append(audio[:max_length])

        # Stack the labels into a tensor
        labels = torch.stack(labels, dim=0)

        # Stack the audio data into a batch tensor
        audio_data_padded = torch.stack(audio_data_padded, dim=0)

        return {"input_values": audio_data_padded, "labels": labels}



# Initialize the dataset and dataloaders
audio_directory = r"audiosets/ontology"
ontology_file = 'ontology.json'

# Initialize dataset and prepare data
dataset = AudioDataset(audio_directory, ontology_file, mid_to_index)

# Now split the dataset into train and test sets (80% train, 20% test)
train_data, test_data = train_test_split(dataset.data, test_size=0.2, random_state=42)

# Initialize train and test datasets using the split data
train_dataset = AudioDataset(audio_directory, ontology_file, mid_to_index)
test_dataset = AudioDataset(audio_directory, ontology_file, mid_to_index)

# Assign the split data to the datasets
train_dataset.data = train_data
test_dataset.data = test_data


def compute_metrics(pred):
    logits, labels = pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = (predictions == labels).mean()
    return {'accuracy': accuracy}

# Training setup
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=3,
    save_steps=10,
    disable_tqdm=False,
    report_to="tensorboard",
    load_best_model_at_end=True,
)

# Initialize the dataset and dataloaders with the custom collate_fn
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)


# Pass the datasets without specifying the dataloaders in Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Start training
trainer.train()




Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [7]:
import glob
import os
import json
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2Processor
import torch
import librosa

# Load class map CSV (mapping from mid to index)
class_map_df = pd.read_csv('yamnet_class_map.csv')

# Create a mapping from mid to index
mid_to_index = class_map_df.set_index('mid').index.tolist()

# Initialize the model and processor
model = Wav2Vec2ForSequenceClassification.from_pretrained('facebook/wav2vec2-large-960h', num_labels=len(class_map))
processor = Wav2Vec2Processor.from_pretrained('facebook/wav2vec2-large-960h')

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("Moved model to GPU if available")
class AudioDataset(Dataset):
    def __init__(self, audio_directory, ontology_file, class_map):
        with open(ontology_file, 'r') as f:
            self.ontology_data = json.load(f)

        self.class_map = class_map
        self.audio_directory = audio_directory
        self.audio_files = glob.glob(os.path.join(self.audio_directory, '**', '*.wav'), recursive=True)
        
        # Populate the dataset by calling prepare_data
        self.data = self.prepare_data()

    def prepare_data(self):
        data = []
        for category in self.ontology_data:
            if "positive_examples" in category:
                category_name = category["name"]
                mid = category["id"]  # Get the mid for the current category

                # Use the mid to get the index from the class_map
                if mid in mid_to_index:
                    label = mid_to_index.index(mid)  # Use the index as the numeric label
                else:
                    label = -1  # Default to -1 if not found

                for audio_file in self.audio_files:
                    if category_name.lower() in audio_file.lower():
                        audio_file = audio_file.replace("\\", "/")
                        data.append({"audio": audio_file, "label": label})
        return data
    
    def load_audio(self, file_path):
        """Load and preprocess audio using Wav2Vec2Processor."""
        try:
            if not os.path.isfile(file_path):
                raise FileNotFoundError(f"WAV file not found: {file_path}")

            # Load audio using librosa and resample to 16kHz
            audio_data, sr = librosa.load(file_path, sr=16000)
            print(f"Successfully loaded audio: {file_path}, shape: {audio_data.shape}, dtype: {audio_data.dtype}")

            # Preprocess using Wav2Vec2Processor
            inputs = processor(audio_data, sampling_rate=sr, return_tensors="pt", padding=True)
            processed_audio = inputs.input_values.squeeze(0)  # Remove batch dimension

            # Check if audio_data is a tensor and not a string
            if isinstance(processed_audio, torch.Tensor):
                print(f"Processed audio data type: {type(processed_audio)}, shape: {processed_audio.shape}")
                return processed_audio
            else:
                print(f"Unexpected processed audio data type: {type(processed_audio)}")
                return None

        except Exception as e:
            print(f"Error loading audio file {file_path}: {e}")
            return None


    def __getitem__(self, idx):
        """Get one item (audio, label) for the dataset."""
        sample = self.data[idx]
        audio_data = self.load_audio(sample["audio"])
        label = sample["label"]

        # Ensure audio_data is valid
        if audio_data is None:
            print(f"Error loading audio at index {idx}, returning dummy data.")
            return torch.zeros(1), torch.tensor(label, dtype=torch.long)

        # Check if audio_data is valid (numpy array or tensor)
        if isinstance(audio_data, (np.ndarray, torch.Tensor)):
            print(f"Successfully loaded audio at index {idx}, shape: {audio_data.shape}, dtype: {audio_data.dtype}")
            
            # Ensure audio_data is not a string or file path
            if isinstance(audio_data, str):
                print(f"Unexpected audio_data type at index {idx}: {type(audio_data)}, returning dummy data.")
                return torch.zeros(1), torch.tensor(label, dtype=torch.long)

            # Return valid audio data
            return audio_data.clone().detach(), torch.tensor(label, dtype=torch.long)
        else:
            print(f"Unexpected audio_data type at index {idx}: {type(audio_data)}, returning dummy data.")
            return torch.zeros(1), torch.tensor(label, dtype=torch.long)


    def __len__(self):
        return len(self.data)


# Initialize the dataset and dataloaders
audio_directory = r"audiosets/ontology"
ontology_file = 'ontology.json'
class_map = pd.read_csv('yamnet_class_map.csv').set_index('display_name').to_dict()['mid']

# Initialize dataset and prepare data
dataset = AudioDataset(audio_directory, ontology_file, class_map)

# Now split the dataset into train and test sets (80% train, 20% test)
train_data, test_data = train_test_split(dataset.data, test_size=0.2, random_state=42)

# Initialize train and test datasets using the split data
train_dataset = AudioDataset(audio_directory, ontology_file, class_map)
test_dataset = AudioDataset(audio_directory, ontology_file, class_map)

# Assign the split data to the datasets
train_dataset.data = train_data
test_dataset.data = test_data

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)

def compute_metrics(pred):
    logits, labels = pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = (predictions == labels).mean()
    return {'accuracy': accuracy}

# Training setup
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=3,
    save_steps=10,
    disable_tqdm=False,
    report_to="tensorboard",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Start training
trainer.train()


Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Moved model to GPU if available


c:\Users\prave\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Successfully loaded audio: audiosets/ontology/Snort_0.wav, shape: (1857411,), dtype: float32
Processed audio data type: <class 'torch.Tensor'>, shape: torch.Size([1857411])
Successfully loaded audio at index 649, shape: torch.Size([1857411]), dtype: torch.float32
Successfully loaded audio: audiosets/ontology/Electronic organ_6.wav, shape: (160125,), dtype: float32
Processed audio data type: <class 'torch.Tensor'>, shape: torch.Size([160125])
Successfully loaded audio at index 360, shape: torch.Size([160125]), dtype: torch.float32
Successfully loaded audio: audiosets/ontology/Blender_6.wav, shape: (159754,), dtype: float32
Processed audio data type: <class 'torch.Tensor'>, shape: torch.Size([159754])
Successfully loaded audio at index 1288, shape: torch.Size([159754]), dtype: torch.float32
Successfully loaded audio: audiosets/ontology/Tubular bells_2.wav, shape: (160125,), dtype: float32
Processed audio data type: <class 'torch.Tensor'>, shape: torch.Size([160125])
Successfully loaded a

TypeError: vars() argument must have __dict__ attribute

In [1]:
import librosa
import torch
from transformers import Wav2Vec2Processor

# Initialize processor
processor = Wav2Vec2Processor.from_pretrained('facebook/wav2vec2-large-960h')

def load_and_preprocess_audio(file_path):
    try:
        # Load audio using librosa and resample to 16kHz (standard for Wav2Vec2)
        audio_data, sr = librosa.load(file_path, sr=16000)
        print(f"Successfully loaded audio: {file_path}, shape: {audio_data.shape}, dtype: {audio_data.dtype}")

        # Process the audio with Wav2Vec2Processor
        inputs = processor(audio_data, sampling_rate=sr, return_tensors="pt", padding=True)
        processed_audio = inputs.input_values.squeeze(0)  # Remove batch dimension

        # Check if the result is a tensor
        print(f"Processed audio data type: {type(processed_audio)}, shape: {processed_audio.shape}")
        return processed_audio

    except Exception as e:
        print(f"Error loading audio file {file_path}: {e}")
        return None

# Test with one audio file
file_path = "audiosets/ontology/Snort_0.wav"  # Use a sample path to one of your .wav files
processed_audio = load_and_preprocess_audio(file_path)

# Check the result
if processed_audio is not None:
    print(f"Audio Tensor: {processed_audio}")
else:
    print("Failed to process audio.")


Successfully loaded audio: audiosets/ontology/Snort_0.wav, shape: (1857411,), dtype: float32
Processed audio data type: <class 'torch.Tensor'>, shape: torch.Size([1857411])
Audio Tensor: tensor([-2.8886e-05, -2.8886e-05, -2.8886e-05,  ..., -2.8886e-05,
        -2.8886e-05, -2.8886e-05])
